<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/Gee_NDVI_pre_postMerida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:

# Celda Final: Autenticación, Inicialización y Prueba de Uso

import ee
from IPython.display import display, Markdown

# ⚠️ Usamos tu ID de proyecto.
PROYECTO_ID = "gen-lang-client-0528470346"

# 1. FORZAR LA AUTENTICACIÓN (pedirá un nuevo enlace y código)
display(Markdown("### ⚠️ Paso 1: Forzar Autenticación"))
display(Markdown("Si no aparece el enlace, reinicia e inténtalo de nuevo."))

# El comando 'ee.Authenticate' debería mostrar el enlace y pedir el código.
ee.Authenticate()

# 2. Inicialización
try:
    display(Markdown("### ✅ Paso 2: Inicializando con Proyecto..."))
    ee.Initialize(project=PROYECTO_ID)

    # Prueba rápida para confirmar que GEE está listo
    test_geometry = ee.Geometry.Point(-71.146, 8.587)
    display(Markdown("✅ **¡ÉXITO! Google Earth Engine está listo para usar.**"))

except Exception as e:
    display(Markdown(f"❌ **Error Fatal:** La inicialización falló. El problema debe ser un permiso faltante en el proyecto **`{PROYECTO_ID}`** dentro de la consola de Google Cloud, a pesar de que la API esté habilitada."))

### ⚠️ Paso 1: Forzar Autenticación

Si no aparece el enlace, reinicia e inténtalo de nuevo.

### ✅ Paso 2: Inicializando con Proyecto...

✅ **¡ÉXITO! Google Earth Engine está listo para usar.**

In [37]:

# 💻 Celda 2: Análisis de NDVI (Versión Final Concatenada para Móviles)

import ee
import geemap
from datetime import datetime

# --- 1. DEFINICIÓN DE PARÁMETROS ---
GEE_ASSET_PATH = 'projects/gen-lang-client-0528470346/assets/MeridaBorde'
merida_borde = ee.FeatureCollection(GEE_ASSET_PATH)

ANOMALY_DATE = '2025-06-14'
DAYS_DELTA = 180

START_PRE = ee.Date(ANOMALY_DATE).advance(-DAYS_DELTA, 'day').format('YYYY-MM-dd').getInfo()
END_PRE = ee.Date(ANOMALY_DATE).format('YYYY-MM-dd').getInfo()
START_POST = ee.Date(ANOMALY_DATE).advance(1, 'day').format('YYYY-MM-dd').getInfo()
END_POST = ee.Date(ANOMALY_DATE).advance(DAYS_DELTA + 1, 'day').format('YYYY-MM-dd').getInfo()

print(f"Periodo de Referencia (Pre-Estrés): {START_PRE} a {END_PRE}")
print(f"Periodo de Estrés (Post-Estrés): {START_POST} a {END_POST}")

# --- 2. FUNCIÓN DE CÁLCULO NDVI ---
def calculate_ndvi(image):
  return image.normalizedDifference(['B8', 'B4']).rename('NDVI') \
      .copyProperties(image, ['system:time_start'])

# --- 3. PROCESAMIENTO DE IMÁGENES SENTINEL-2 (Sin saltos de línea para evitar IndentationError) ---
def get_average_ndvi(start_date, end_date):
  # 🚨 Todo en una línea para evitar errores de indentación al copiar:
  collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date).filterBounds(merida_borde).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 29))

  ndvi_collection = collection.map(calculate_ndvi)
  return ndvi_collection.median().clip(merida_borde)

# Obtener imágenes promedio
ndvi_pre = get_average_ndvi(START_PRE, END_PRE)
ndvi_post = get_average_ndvi(START_POST, END_POST)

# --- 4. VISUALIZACIÓN ---
ndvi_vis = {
    'min': 0.1, 'max': 0.9,
    'palette': ['FF0000', 'FF9900', 'FFFF00', '00FF00']
}

center_coords = merida_borde.geometry().centroid().coordinates().getInfo()

Map = geemap.Map(center=[center_coords[1], center_coords[0]], zoom=10)

Map.addLayer(ndvi_pre, ndvi_vis, f'NDVI Pre-Estrés ({START_PRE} a {END_PRE})')
Map.addLayer(ndvi_post, ndvi_vis, f'NDVI Post-Estrés ({START_POST} a {END_POST})')
Map.addLayer(merida_borde, {'color': 'FFFFFF', 'fillColor': '00000000'}, 'Límite de Mérida')

Map.add_legend(title="Salud Vegetal (NDVI)", legend_dict={
    'Malo (0.1)': 'FF0000', 'Medio (0.5)': 'FFFF00', 'Bueno (0.9)': '00FF00'
})

print("\n✅ Generando mapas de NDVI (Pre vs Post Anomalía)...")
Map

Periodo de Referencia (Pre-Estrés): 2024-12-16 a 2025-06-14
Periodo de Estrés (Post-Estrés): 2025-06-15 a 2025-12-12

✅ Generando mapas de NDVI (Pre vs Post Anomalía)...


Map(center=[8.508124203391839, -71.26270084496359], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:

# 💻 Celda 3: Cuantificación del Estrés Vegetal (Análisis de la diferencia de NDVI)

import ee
from datetime import datetime

# Aseguramos que los parámetros de fecha estén definidos (se asume que Celda 2 se ejecutó)
ANOMALY_DATE = '2025-06-14'
DAYS_DELTA = 180
START_PRE = ee.Date(ANOMALY_DATE).advance(-DAYS_DELTA, 'day').format('YYYY-MM-dd').getInfo()
END_PRE = ee.Date(ANOMALY_DATE).format('YYYY-MM-dd').getInfo()
START_POST = ee.Date(ANOMALY_DATE).advance(1, 'day').format('YYYY-MM-dd').getInfo()
END_POST = ee.Date(ANOMALY_DATE).advance(DAYS_DELTA + 1, 'day').format('YYYY-MM-dd').getInfo()

# Las variables ndvi_pre, ndvi_post y merida_borde deben existir del paso anterior
if 'ndvi_pre' in locals() and 'ndvi_post' in locals():
    try:
        # --- 1. Calcular el NDVI promedio para el periodo Pre-Estrés ---
        # reduceRegion calcula la media de los píxeles dentro del polígono.
        stats_pre = ndvi_pre.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=merida_borde.geometry(),
            scale=30, # Resolución de 30 metros (Sentinel-2)
            maxPixels=1e9,
            bestEffort=True
        ).get('NDVI').getInfo()

        # --- 2. Calcular el NDVI promedio para el periodo Post-Estrés ---
        stats_post = ndvi_post.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=merida_borde.geometry(),
            scale=30,
            maxPixels=1e9,
            bestEffort=True
        ).get('NDVI').getInfo()

        # --- 3. Cuantificar el Estrés ---

        # Aseguramos que los valores sean numéricos (float)
        mean_ndvi_pre = float(stats_pre)
        mean_ndvi_post = float(stats_post)

        # Calcular la diferencia porcentual
        if mean_ndvi_pre is not None and mean_ndvi_pre != 0:
            diferencia_pct = ((mean_ndvi_post - mean_ndvi_pre) / mean_ndvi_pre) * 100
        else:
            diferencia_pct = 0

        print("--- 📊 Resultados Cuantitativos del Estrés ---")
        print(f"NDVI Promedio PRE-Estrés ({START_PRE} a {END_PRE}): {mean_ndvi_pre:.4f}")
        print(f"NDVI Promedio POST-Estrés ({START_POST} a {END_POST}): {mean_ndvi_post:.4f}")

        print("\n--- 📈 Interpretación de Resultados ---")
        if diferencia_pct < -1:
            print(f"Resultado: 📉 Estrés Vegetal Significativo. El NDVI promedio se redujo en {-diferencia_pct:.2f}%.")
            print("Interpretación: La disminución del NDVI indica una posible afectación o daño en la vegetación debido al frío extremo.")
        elif diferencia_pct > 1:
            print(f"Resultado: 🌿 Mejora de la Vegetación. El NDVI promedio aumentó en {diferencia_pct:.2f}%.")
            print("Interpretación: El aumento del NDVI indica que la salud vegetal mejoró, o que el frío no tuvo un impacto medible.")
        else:
            print(f"Resultado: Estabilidad. Cambio porcentual de {diferencia_pct:.2f}%.")
            print("Interpretación: El evento de frío no causó un cambio significativo en el área.")

    except Exception as e:
         print(f"❌ Error al calcular reduceRegion. La clave 'NDVI' no se encontró o hubo un error de conexión: {e}")

else:
    print("❌ No se encontraron las variables 'ndvi_pre' o 'ndvi_post'. Asegúrate de que la Celda 2 se ejecutó correctamente.")

--- 📊 Resultados Cuantitativos del Estrés ---
NDVI Promedio PRE-Estrés (2024-12-16 a 2025-06-14): 0.4882
NDVI Promedio POST-Estrés (2025-06-15 a 2025-12-12): 0.5074

--- 📈 Interpretación de Resultados ---
Resultado: 🌿 Mejora de la Vegetación. El NDVI promedio aumentó en 3.93%.
Interpretación: El aumento del NDVI indica que la salud vegetal mejoró, o que el frío no tuvo un impacto medible.
